# [TransformersのQuestion Answeringを用いた係り受け解析器](https://koichiyasuoka.github.io/deplacy/demo/2022-06-17/)
## モデル簡易作成ツール(slowトークナイザ向け)


必要なパッケージと各conlluを準備

In [ ]:
import os
url="https://github.com/UniversalDependencies/UD_Japanese-GSDLUW"
d=os.path.basename(url)
!test -d $d || git clone --depth=1 $url
!for F in train dev test ; do cat $d/*-$$F.conllu > $$F.conllu ; done
url="https://github.com/huggingface/transformers"
d=os.path.basename(url)
!pip install $d pytokenizations ufal.chu-liu-edmonds datasets seqeval
!pip install fugashi unidic-lite
!test -d $d || git clone --depth=1 -b `pip list | sed -n "s/^$d */v/p"` $url
url="https://universaldependencies.org/conll18/conll18_ud_eval.py"
f=os.path.basename(url)
!test -f $f || curl -LO $url
!test -f $f || curl -LO $url
url="https://raw.githubusercontent.com/KoichiYasuoka/deplacy"
url+="/master/demo/2022-06-17/UD-KyotsuTest2022Kokugo"
!test -f question1-1.conllu || curl -LO $url/question1-1.conllu
!test -f question1-2.conllu || curl -LO $url/question1-2.conllu

qa-model作成(HEAD返答モデル)

In [ ]:
import tokenizations,json
from transformers import AutoTokenizer,AutoConfig
src="KoichiYasuoka/deberta-base-japanese-unidic"
m,n=AutoTokenizer.from_pretrained(src).mask_token,0
for f in ["train","dev","test"]:
  u,v,w=[],None,[]
  with open(f+".conllu","r",encoding="utf-8") as r:
    for s in r.read().split("\n"):
      if s.startswith("# text = "):
        v=s[9:]
      elif s=="":
        if v and w!=[]:
          v2w,w2v=tokenizations.get_alignments(v,[x[1] for x in w])
          for i,t in enumerate([] if (w2v+[[]]).index([])<len(w2v) else w):
            n,c,h=n+1,v[0:w2v[i][0]]+m+v[w2v[i][-1]+1:],int(t[6])-1
            s=w2v[i if h<0 else h][0]+(len(m)-len(t[1]) if i<h else 0)
            u.append({"context":c,"qas":[{"id":n,"question":t[1],
              "answers":[{"text":m if h<0 else w[h][1],"answer_start":s}]}]})
            if f=="train":
              if [1 for x in w if x[1]==t[1]]==[1]:
                n,j=n+1,i if h<0 else h
                u.append({"context":v,"qas":[{"id":n,"question":t[1],
                  "answers":[{"text":w[j][1],"answer_start":w2v[j][0]}]}]})
        v,w=None,[]
      else:
        t=s.split("\t")
        if len(t)==10 and t[0].isdecimal():
          w.append(t)
  with open(f+".json","w",encoding="utf-8") as w:
    json.dump({"data":[{"title":"UD","paragraphs":u}]},w,ensure_ascii=False)
t=AutoConfig.from_pretrained(src).model_type
s=" ".join(["transformers/examples/legacy/question-answering/run_squad.py",
  "--model_name_or_path="+src,"--model_type="+t,
  "--output_dir=qa-model --overwrite_output_dir --save_steps=5000",
  "--per_gpu_train_batch_size=16 --per_gpu_eval_batch_size=16",
  "--train_file=train.json --predict_file=dev.json",
  "--do_train --do_eval --learning_rate=5e-05 --num_train_epochs=3"])
!python $s

qa-model/deprel作成(DEPREL付与モデル)

In [ ]:
src="KoichiYasuoka/deberta-base-japanese-unidic"
class UPOSDataset(object):
  def __init__(self,conllu,tokenizer,fields=[3]):
    self.ids,self.upos=[],[]
    label,cls,sep=set(),tokenizer.cls_token_id,tokenizer.sep_token_id
    with open(conllu,"r",encoding="utf-8") as r:
      form,upos=[],[]
      for t in r:
        w=t.split("\t")
        if len(w)==10 and w[0].isdecimal():
          form.append(w[1])
          upos.append("|".join(w[i] for i in fields))
        elif t.strip()=="" and form!=[]:
          v,u=tokenizer(form,add_special_tokens=False)["input_ids"],[]
          for x,y in zip(v,upos):
            u.extend(["B-"+y]*min(len(x),1)+["I-"+y]*(len(x)-1))
          if len(u)>tokenizer.model_max_length-4:
            self.ids.append(sum(v,[])[0:tokenizer.model_max_length-2])
            self.upos.append(u[0:tokenizer.model_max_length-2])
          elif len(u)>0:
            self.ids.append([cls]+sum(v,[])+[sep])
            self.upos.append([u[0]]+u+[u[0]])
          label=set(sum([self.upos[-1],list(label)],[]))
          form,upos=[],[]
    self.label2id={l:i for i,l in enumerate(sorted(label))}
  def __call__(*args):
    label=set(sum([list(t.label2id) for t in args],[]))
    lid={l:i for i,l in enumerate(sorted(label))}
    for t in args:
      t.label2id=lid
    return lid
  __len__=lambda self:len(self.ids)
  __getitem__=lambda self,i:{"input_ids":self.ids[i],
    "labels":[self.label2id[t] for t in self.upos[i]]}
from transformers import (AutoTokenizer,AutoModelForTokenClassification,
  AutoConfig,DataCollatorForTokenClassification,TrainingArguments,Trainer)
tkz=AutoTokenizer.from_pretrained(src)
trainDS=UPOSDataset("train.conllu",tkz,[7])
devDS=UPOSDataset("dev.conllu",tkz,[7])
testDS=UPOSDataset("test.conllu",tkz,[7])
lid=trainDS(devDS,testDS)
cfg=AutoConfig.from_pretrained(src,num_labels=len(lid),label2id=lid,
  id2label={i:l for l,i in lid.items()})
arg=TrainingArguments(num_train_epochs=3,per_device_train_batch_size=16,
  output_dir="qa-model",overwrite_output_dir=True,save_total_limit=2)
trn=Trainer(args=arg,data_collator=DataCollatorForTokenClassification(tkz),
  model=AutoModelForTokenClassification.from_pretrained(src,config=cfg),
  train_dataset=trainDS,eval_dataset=devDS)
trn.train()
trn.save_model("qa-model/deprel")
tkz.save_pretrained("qa-model/deprel")

qa-model/tagger作成(UPOS・FEATS付与モデル)

In [ ]:
src="KoichiYasuoka/deberta-base-japanese-unidic"
class UPOSDataset(object):
  def __init__(self,conllu,tokenizer,fields=[3]):
    self.ids,self.upos=[],[]
    label,cls,sep=set(),tokenizer.cls_token_id,tokenizer.sep_token_id
    with open(conllu,"r",encoding="utf-8") as r:
      form,upos=[],[]
      for t in r:
        w=t.split("\t")
        if len(w)==10 and w[0].isdecimal():
          form.append(w[1])
          upos.append("|".join(w[i] for i in fields))
        elif t.strip()=="" and form!=[]:
          v,u=tokenizer(form,add_special_tokens=False)["input_ids"],[]
          for x,y in zip(v,upos):
            u.extend(["B-"+y]*min(len(x),1)+["I-"+y]*(len(x)-1))
          if len(u)>tokenizer.model_max_length-4:
            self.ids.append(sum(v,[])[0:tokenizer.model_max_length-2])
            self.upos.append(u[0:tokenizer.model_max_length-2])
          elif len(u)>0:
            self.ids.append([cls]+sum(v,[])+[sep])
            self.upos.append([u[0]]+u+[u[0]])
          label=set(sum([self.upos[-1],list(label)],[]))
          form,upos=[],[]
    self.label2id={l:i for i,l in enumerate(sorted(label))}
  def __call__(*args):
    label=set(sum([list(t.label2id) for t in args],[]))
    lid={l:i for i,l in enumerate(sorted(label))}
    for t in args:
      t.label2id=lid
    return lid
  __len__=lambda self:len(self.ids)
  __getitem__=lambda self,i:{"input_ids":self.ids[i],
    "labels":[self.label2id[t] for t in self.upos[i]]}
from transformers import (AutoTokenizer,AutoModelForTokenClassification,
  AutoConfig,DataCollatorForTokenClassification,TrainingArguments,Trainer)
tkz=AutoTokenizer.from_pretrained(src)
trainDS=UPOSDataset("train.conllu",tkz,[3,5])
devDS=UPOSDataset("dev.conllu",tkz,[3,5])
testDS=UPOSDataset("test.conllu",tkz,[3,5])
lid=trainDS(devDS,testDS)
cfg=AutoConfig.from_pretrained(src,num_labels=len(lid),label2id=lid,
  id2label={i:l for l,i in lid.items()})
arg=TrainingArguments(num_train_epochs=3,per_device_train_batch_size=64,
  output_dir="qa-model",overwrite_output_dir=True,save_total_limit=2)
trn=Trainer(args=arg,data_collator=DataCollatorForTokenClassification(tkz),
  model=AutoModelForTokenClassification.from_pretrained(src,config=cfg),
  train_dataset=trainDS,eval_dataset=devDS)
trn.train()
trn.save_model("qa-model/tagger")
tkz.save_pretrained("qa-model/tagger")

評価とテスト

In [ ]:
from transformers import (AutoTokenizer,AutoModelForQuestionAnswering,
  AutoModelForTokenClassification,AutoConfig,TokenClassificationPipeline)
class TaggerPipeline(TokenClassificationPipeline):
  def __call__(self,text):
    d=super().__call__(text)
    if len(d)>0 and ("start" not in d[0] or d[0]["start"]==None):
      import tokenizations
      v=[x["word"].replace(" ","") for x in d]
      a2b,b2a=tokenizations.get_alignments(v,text)
      for i,t in enumerate(a2b):
        s,e=(0,0) if t==[] else (t[0],t[-1]+1)
        if v[i].startswith(self.tokenizer.unk_token):
          s=([[-1]]+[x for x in a2b[0:i] if x>[]])[-1][-1]+1
        if v[i].endswith(self.tokenizer.unk_token):
          e=([x for x in a2b[i+1:] if x>[]]+[[len(text)]])[0][0]
        d[i]["start"],d[i]["end"]=s,e
    return d
class TransformersSlowUD(object):
  def __init__(self,bert):
    import os
    self.tokenizer=AutoTokenizer.from_pretrained(bert)
    self.model=AutoModelForQuestionAnswering.from_pretrained(bert)
    x=AutoModelForTokenClassification.from_pretrained
    if os.path.isdir(bert):
      d,t=x(os.path.join(bert,"deprel")),x(os.path.join(bert,"tagger"))
    else:
      from transformers.file_utils import hf_bucket_url
      c=AutoConfig.from_pretrained(hf_bucket_url(bert,"deprel/config.json"))
      d=x(hf_bucket_url(bert,"deprel/pytorch_model.bin"),config=c)
      s=AutoConfig.from_pretrained(hf_bucket_url(bert,"tagger/config.json"))
      t=x(hf_bucket_url(bert,"tagger/pytorch_model.bin"),config=s)
    self.deprel=TaggerPipeline(model=d,tokenizer=self.tokenizer,
      aggregation_strategy="simple")
    self.tagger=TaggerPipeline(model=t,tokenizer=self.tokenizer)
  def __call__(self,text):
    import numpy,torch,ufal.chu_liu_edmonds
    w=[(t["start"],t["end"],t["entity_group"]) for t in self.deprel(text)]
    z,n={t["start"]:t["entity"].split("|") for t in self.tagger(text)},len(w)
    r,m=[text[s:e] for s,e,p in w],numpy.full((n+1,n+1),numpy.nan)
    v,c=self.tokenizer(r,add_special_tokens=False)["input_ids"],[]
    for i,t in enumerate(v):
      q=[self.tokenizer.cls_token_id]+t+[self.tokenizer.sep_token_id]
      c.append([q]+v[0:i]+[[self.tokenizer.mask_token_id]]+v[i+1:]+[[q[-1]]])
    b=[[len(sum(x[0:j+1],[])) for j in range(len(x))] for x in c]
    d=self.model(input_ids=torch.tensor([sum(x,[]) for x in c]),
      token_type_ids=torch.tensor([[0]*x[0]+[1]*(x[-1]-x[0]) for x in b]))
    s,e=d.start_logits.tolist(),d.end_logits.tolist()
    for i in range(n):
      for j in range(n):
        m[i+1,0 if i==j else j+1]=s[i][b[i][j]]+e[i][b[i][j+1]-1]
    h=ufal.chu_liu_edmonds.chu_liu_edmonds(m)[0]
    if [0 for i in h if i==0]!=[0]:
      i=([p for s,e,p in w]+["root"]).index("root")
      j=i+1 if i<n else numpy.nanargmax(m[:,0])
      m[0:j,0]=m[j+1:,0]=numpy.nan
      h=ufal.chu_liu_edmonds.chu_liu_edmonds(m)[0]
    u="# text = "+text.replace("\n"," ")+"\n"
    for i,(s,e,p) in enumerate(w,1):
      p="root" if h[i]==0 else "dep" if p=="root" else p
      u+="\t".join([str(i),r[i-1],"_",z[s][0][2:],"_","|".join(z[s][1:]),
        str(h[i]),p,"_","_" if i<n and w[i][0]<e else "SpaceAfter=No"])+"\n"
    return u+"\n"

nlp=TransformersSlowUD("qa-model")
for f in ["dev.conllu","test.conllu","question1-1.conllu","question1-2.conllu"]:
  with open(f,"r",encoding="utf-8") as r:
    with open("result-"+f,"w",encoding="utf-8") as w:
      for s in r.read().split("\n"):
        if s.startswith("# text = "):
          w.write(nlp(s[9:]))
  print("***",f)
  !python conll18_ud_eval.py -v $f result-$f